In [30]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm

In [33]:
from pathlib import Path

# Get the current working directory
current_path = Path.cwd()

# Get the parent directory
parent_path = current_path
data_paths = parent_path / 'datas'  

print(data_paths)
#
train_path = data_paths / 'train.csv'
test_path = data_paths / 'test.csv'

train_path_ts = data_paths / 'series_train.parquet'
test_path_ts = data_paths / 'series_test.parquet'

/home/koko/Bureau/projets/kaggle/cmi_probleamtic_use/CMI_problematic_internet_use/datas


In [31]:
# Load datasets

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [32]:
train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
from concurrent.futures import ThreadPoolExecutor

def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df
        
train_ts = load_time_series(train_path_ts)
test_ts = load_time_series(test_path_ts)

100%|██████████| 2/2 [00:00<00:00, 11.79it/s]


In [47]:
single_data = pd.read_parquet(data_paths / 'series_train.parquet/id=93c06d4c/part-0.parquet')

In [48]:
single_data.head()

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,0.755956,-0.064450,-0.641633,0.038805,-39.163845,0.0,10.000000,4182.000000,45240000000000,2,4,21.0
1,1,0.377183,0.524232,-0.081910,0.126356,-3.364063,0.0,4.000000,4181.666504,45245000000000,2,4,21.0
2,2,0.243445,0.589751,0.538799,0.069743,38.590385,0.0,9.000000,4181.333496,45250000000000,2,4,21.0
3,3,0.885580,0.225155,-0.199520,0.063883,-12.149102,0.0,14.000000,4181.000000,45255000000000,2,4,21.0
4,4,0.184744,0.071157,-0.090001,0.160263,-3.711064,0.0,5.333333,4180.666504,45260000000000,2,4,21.0


In [44]:
train_ts

,stat_0,stat_1,stat_2,stat_3,stat_4,stat_5,stat_6,stat_7,stat_8,stat_9,...,stat_87,stat_88,stat_89,stat_90,stat_91,stat_92,stat_93,stat_94,stat_95,id
0,927.0,927.0,927.0,927.0,927.0,927.0,927.0,927.0,927.0,927.0,...,0.248611,76.602829,0.0,1070.599976,4152.000000,7.704500e+13,7.0,4.0,293.0,e1ea8dd7
1,397788.0,397788.0,397788.0,397788.0,397788.0,397788.0,397788.0,397788.0,397788.0,397788.0,...,3.170410,89.264389,1.0,2538.000000,4175.000000,8.639500e+13,7.0,3.0,34.0,059eed01
2,321996.0,321996.0,321996.0,321996.0,321996.0,321996.0,321996.0,321996.0,321996.0,321996.0,...,4.391237,88.928375,1.0,2644.666748,4066.000000,8.639500e+13,7.0,2.0,22.0,de3ecca9
3,397812.0,397812.0,397812.0,397812.0,397812.0,397812.0,397812.0,397812.0,397812.0,397812.0,...,3.676018,89.532845,1.0,2643.666748,4166.000000,8.639500e+13,7.0,4.0,160.0,9130a22e
4,365483.0,365483.0,365483.0,365483.0,365483.0,365483.0,365483.0,365483.0,365483.0,365483.0,...,4.488843,89.727219,1.0,2542.399902,4187.000000,8.639500e+13,7.0,4.0,26.0,a6969e1a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,100786.0,100786.0,100786.0,100786.0,100786.0,100786.0,100786.0,100786.0,100786.0,100786.0,...,4.089762,89.738800,0.0,2632.800049,4170.000000,8.628500e+13,7.0,2.0,30.0,5c2c136d
992,390924.0,390924.0,390924.0,390924.0,390924.0,390924.0,390924.0,390924.0,390924.0,390924.0,...,2.226662,88.689301,1.0,142.399994,4193.000000,8.639500e+13,7.0,4.0,23.0,9e964060
993,23722.0,23722.0,23722.0,23722.0,23722.0,23722.0,23722.0,23722.0,23722.0,23722.0,...,1.170792,89.298759,0.0,1314.000000,4186.000000,8.639500e+13,7.0,3.0,62.0,c9902f48
994,380339.0,380339.0,380339.0,380339.0,380339.0,380339.0,380339.0,380339.0,380339.0,380339.0,...,4.396735,89.020294,0.0,2462.600098,4183.333496,8.639500e+13,7.0,4.0,38.0,81a4b826


# Nan Values
